# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [48]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [49]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [50]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
print(len(full_data_rows_list))

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [51]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [52]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [53]:
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS music
        WITH REPLICATION=
        { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [54]:
try:
    session.set_keyspace('music')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




We will create a table to retrieve the data by its session and its item for the song info.

In [55]:
try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS song_info_by_session(
            session_id INT,
            item_in_session INT,
            artist TEXT,
            song TEXT,
            song_length DECIMAL,
            PRIMARY KEY(session_id, item_in_session)
        )
    """)
except Exception as e:
    print(e)

In [56]:
file = 'event_datafile_new.csv'

event_df = pd.read_csv(file)
event_df.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Mohammad,M,0,Rodriguez,277.15873,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Horn Concerto No. 4 in E flat K495: II. Romanc...,88
1,Jimi Hendrix,Mohammad,M,1,Rodriguez,239.82975,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Woodstock Inprovisation,88
2,Building 429,Mohammad,M,2,Rodriguez,300.61669,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Majesty (LP Version),88
3,The B-52's,Gianna,F,0,Jones,321.54077,free,"New York-Newark-Jersey City, NY-NJ-PA",107,Love Shack,38
4,Die Mooskirchner,Gianna,F,1,Jones,169.29914,free,"New York-Newark-Jersey City, NY-NJ-PA",107,Frisch und g'sund,38


In [57]:
query = """
    INSERT INTO song_info_by_session(
        session_id, 
        item_in_session, 
        artist, 
        song, 
        song_length
    ) VALUES(%s, %s, %s, %s, %s)
"""

for index, row in event_df.iterrows():
    session.execute(
        query, 
        (row.sessionId, row.itemInSession, row.artist, row.song, row.length)
    )

#### Do a SELECT to verify that the data have been inserted into each table

In [58]:
query = """
    SELECT * FROM song_info_by_session 
        WHERE session_id = 338 AND item_in_session = 4
"""
try:
    rows_df = pd.DataFrame(list(session.execute(query)))
except Exception as e:
    print(e)

In [59]:
rows_df

,session_id,item_in_session,artist,song,song_length
0,338,4,Faithless,Music Matters (Mark Knight Dub),495.3073


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

We will create a table to retrieve data by its user, session and item for the user and song info.

In [60]:
try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS user_song_info_by_session(
            user_id INT,
            session_id INT,
            item_in_session INT,
            artist TEXT,
            song TEXT,
            first_name TEXT,
            last_name TEXT,
            PRIMARY KEY((user_id, session_id), item_in_session)
        )
    """)
except Exception as e:
    print(e)

In [61]:
query = """
    INSERT INTO user_song_info_by_session(
        user_id,
        session_id,
        item_in_session,
        artist,
        song,
        first_name,
        last_name
    ) VALUES(%s, %s, %s, %s, %s, %s, %s)
"""

for index, row in event_df.iterrows():
    session.execute(
        query, 
        (row.userId, row.sessionId, row.itemInSession, row.artist, row.song, row.firstName, row.lastName)
    )

In [62]:
query = """
    SELECT * FROM user_song_info_by_session 
        WHERE user_id = 10 AND session_id = 182
"""
try:
    rows_df = pd.DataFrame(list(session.execute(query)))
except Exception as e:
    print(e)

In [63]:
rows_df

,user_id,session_id,item_in_session,artist,first_name,last_name,song
0,10,182,0,Down To The Bone,Sylvie,Cruz,Keep On Keepin' On
1,10,182,1,Three Drives,Sylvie,Cruz,Greece 2000
2,10,182,2,Sebastien Tellier,Sylvie,Cruz,Kilometer
3,10,182,3,Lonnie Gordon,Sylvie,Cruz,Catch You Baby (Steve Pitron & Max Sanna Radio...


We will create a table to retrieve songs played by users for their user info.

In [64]:
try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS song_play_by_user(
            song TEXT,
            user_id INT,
            first_name TEXT,
            last_name TEXT,
            PRIMARY KEY(song, user_id)
        )
    """)
except Exception as e:
    print(e)

In [65]:
query = """
    INSERT INTO song_play_by_user(
        song,
        user_id,
        first_name,
        last_name
    ) VALUES(%s, %s, %s, %s)
"""

for index, row in event_df.iterrows():
    session.execute(
        query, 
        (row.song, row.userId, row.firstName, row.lastName)
    )

In [66]:
query = """
    SELECT * FROM song_play_by_user 
        WHERE song = 'All Hands Against His Own'
"""
try:
    rows_df = pd.DataFrame(list(session.execute(query)))
except Exception as e:
    print(e)

In [67]:
rows_df

,song,user_id,first_name,last_name
0,All Hands Against His Own,29,Jacqueline,Lynch
1,All Hands Against His Own,80,Tegan,Levine
2,All Hands Against His Own,95,Sara,Johnson


### Drop the tables before closing out the sessions

In [68]:
session.execute("drop table song_info_by_session")
session.execute("drop table user_song_info_by_session")
session.execute("drop table song_play_by_user")

### Close the session and cluster connection¶

In [69]:
session.shutdown()
cluster.shutdown()